In [1]:
import pandas as pd

afi_df = pd.read_csv("C:\\Users\\abaker\\OneDrive - Factory Direct of Edison\\Desktop\\textgen-pipeline\\data\\bronze\\api_entities.csv")

C:\Users\abaker\AppData\Local\Temp\ipykernel_29376\617524382.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  afi_df = pd.read_csv("C:\\Users\\abaker\\OneDrive - Factory Direct of Edison\\Desktop\\textgen-pipeline\\data\\bronze\\api_entities.csv")


In [2]:
# Apply the WHERE filter
filtered_df = afi_df[afi_df['itemType'].isin(['Component'])]

# Compute DENSE_RANK over seriesId
filtered_df['seriesRank'] = filtered_df['seriesId'].rank(method='dense').astype(int)

# Select columns similar to SQL output
afi_result = (
    filtered_df[['seriesId', 'itemSeriesName','sku', 'status', 'friendlyDescription', 'seriesRank']]
    .sort_values(by='seriesRank', ascending=True)
    .reset_index(drop=True)
)

C:\Users\abaker\AppData\Local\Temp\ipykernel_29376\3666099613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['seriesRank'] = filtered_df['seriesId'].rank(method='dense').astype(int)


In [4]:
import pandas as pd

# Load your data
df = pd.read_csv("C:\\Users\\abaker\\OneDrive - Factory Direct of Edison\\Desktop\\textgen-pipeline\\data\\bronze\\via_tagging_translator.csv")

# Split ComponentsQTY by commas and explode
df_exploded = df.assign(ComponentsQTY=df['ComponentsQTY'].str.split(';')) \
                .explode('ComponentsQTY')

# Clean up spaces
df_exploded['ComponentsQTY'] = df_exploded['ComponentsQTY'].str.strip()

# Split into SKU and Quantity
df_exploded[['ComponentSKU', 'ComponentQty']] = df_exploded['ComponentsQTY'].str.split(',', n=1, expand=True)

# Convert quantity to integer
df_exploded['ComponentQty'] = pd.to_numeric(df_exploded['ComponentQty'], errors='coerce')

# Drop the original ComponentsQTY if you want
df_exploded = df_exploded.drop(columns=['ComponentsQTY'])

df_exploded = df_exploded[["Series ID","SeriesName","FDE SKU","Description","ComponentSKU","ComponentQty","Status"]].dropna(subset=['FDE SKU'])

df_exploded['seriesRank'] = df_exploded['FDE SKU'].rank(method='dense').astype(int)

result_via_exploded = (
    df_exploded.query("Status == 'Current'")[["Series ID","SeriesName","Description","ComponentSKU","ComponentQty",'seriesRank']]
    .sort_values(by='seriesRank', ascending=True)
    .reset_index(drop=True)
)

merged_df = pd.merge(result_via_exploded, afi_result[['sku', 'friendlyDescription']], left_on='ComponentSKU', right_on='sku',how = "left").drop(columns=['sku'])

merged_df.to_csv('kit_classification_training_data.csv', index=False)
